In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tqdm.auto import tqdm
from timeit import default_timer as timer

import warnings
warnings.filterwarnings("ignore")

In [2]:
# If running in Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
output_path = 'output/models/'

In [4]:
ls

drive/  sample_data/


In [5]:
cd drive/MyDrive/RecSys_KB/Aleksis

/content/drive/MyDrive/RecSys_KB/Aleksis


In [6]:
movies = pd.read_csv('ml-latest-small/movies.csv')
ratings = pd.read_csv('ml-latest-small/ratings.csv')

In [7]:
movies.head()

,movieId,title,genres,abstracts,abstracts_count,abstract_embeddings
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story (1995) genres: Adventure|Animation|C...,0,[-3.30134444e-02 -2.01094374e-02 7.14708790e-...
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji is a 1995 American fantasy adventure f...,1,[-1.18304212e-02 6.41161203e-02 9.13541857e-...
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men is a 1995 American romantic c...,1,[-4.35247384e-02 -2.99240090e-02 -1.32443719e-...
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale (1995) genres: Comedy|Drama|...,0,[-6.43825084e-02 -6.76364526e-02 -1.91318002e-...
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II (1995) genres: Comedy,0,[-4.51376252e-02 -5.00504188e-02 -1.56385768e-...


In [11]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [12]:
print("Any null values in movies:\n" ,movies.isnull().sum())
print("Any null values in ratings train:\n ",ratings.isnull().sum())

Any null values in movies:
 movieId                0
title                  0
genres                 0
abstracts              0
abstracts_count        0
abstract_embeddings    0
dtype: int64
Any null values in ratings train:
  userId       0
movieId      0
rating       0
timestamp    0
dtype: int64


In [13]:
len(movies)

9742

In [14]:
len(movies['title'].unique())

9737

In [15]:
ratings['rating'].unique()

array([4. , 5. , 3. , 2. , 1. , 4.5, 3.5, 2.5, 0.5, 1.5])

In [16]:
ratings['rating'].value_counts(ascending=False)

rating
4.0    26818
3.0    20047
5.0    13211
3.5    13136
4.5     8551
2.0     7551
2.5     5550
1.0     2811
1.5     1791
0.5     1370
Name: count, dtype: int64

In [17]:
ratings['userId'].value_counts(ascending=True)

userId
442      20
406      20
147      20
194      20
569      20
       ... 
274    1346
448    1864
474    2108
599    2478
414    2698
Name: count, Length: 610, dtype: int64

In [18]:
ratings['movieId'].value_counts(ascending=True)

movieId
163981      1
142602      1
142961      1
5706        1
5468        1
         ... 
2571      278
593       279
296       307
318       317
356       329
Name: count, Length: 9724, dtype: int64

In [19]:
movie_ratings_count = ratings['movieId'].value_counts(ascending=True)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(ratings[['userId', 'movieId']], ratings[['rating']], stratify=ratings[['rating']], test_size=0.20, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify=y_train, test_size=0.10, random_state=42)

In [22]:
users_train = X_train.userId.unique()
movies_train = X_train.movieId.unique()

In [23]:
movies_with_enough_ratings = list(filter(lambda x: movie_ratings_count[x] >= 5, movies_train))

In [24]:
len(movies_with_enough_ratings)

3649

In [25]:
userid2idx = {o:i + 1 for i,o in enumerate(users_train)}
movieid2idx = {o:i + 1 for i,o in enumerate(movies_with_enough_ratings)}

In [26]:
len(userid2idx), len(movieid2idx)

(610, 3649)

In [27]:
idx2userid = {v:k  for k,v in userid2idx.items()}
idx2movieid = {v:k  for k,v in movieid2idx.items()}

In [28]:
userid2idx[-1] = 0 # unk user
movieid2idx[-1] = 0 # unk movie

In [29]:
def get_indexes(data):
    data['userId'] = data['userId'].apply(lambda x: userid2idx.get(x, 0))
    data['movieId'] = data['movieId'].apply(lambda x: movieid2idx.get(x, 0))

In [30]:
movies

,movieId,title,genres,abstracts,abstracts_count,abstract_embeddings
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story (1995) genres: Adventure|Animation|C...,0,[-3.30134444e-02 -2.01094374e-02 7.14708790e-...
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji is a 1995 American fantasy adventure f...,1,[-1.18304212e-02 6.41161203e-02 9.13541857e-...
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men is a 1995 American romantic c...,1,[-4.35247384e-02 -2.99240090e-02 -1.32443719e-...
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale (1995) genres: Comedy|Drama|...,0,[-6.43825084e-02 -6.76364526e-02 -1.91318002e-...
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II (1995) genres: Comedy,0,[-4.51376252e-02 -5.00504188e-02 -1.56385768e-...
...,...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,The Butler Bulldogs men's basketball team repr...,2,[ 3.13083529e-02 1.48075968e-02 3.68007384e-...
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,"No Game No Life (Japanese: ノーゲーム・ノーライフ, Hepbur...",1,[-1.92668010e-02 -4.74059843e-02 -8.06948766e-...
9739,193585,Flint (2017),Drama,Flint is a sedimentary cryptocrystalline form ...,1,[-5.53447567e-02 -1.97313577e-02 3.11960699e-...
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,"Bungo Stray Dogs (Japanese: 文豪ストレイドッグス, Hepbur...",2,[-1.11603690e-03 -6.92510679e-02 1.85637157e-...


In [31]:
movies['abstracts'].isna().any()

False

In [32]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [33]:
movie2abstract_embed = {}

In [34]:
import pickle
with open('movie2abstract_embed.pickle', 'rb') as f:
    movie2abstract_embed = pickle.load(f)

In [35]:
movie2abstract_embed[1]

array([-3.30134444e-02, -2.01094374e-02,  7.14708790e-02, -1.27697820e-02,
       -9.81519651e-03,  4.56205383e-02,  3.72679681e-02, -8.54881015e-03,
        2.10595690e-03,  3.40597406e-02, -6.33732090e-03,  4.50791754e-02,
       -1.71999559e-02,  7.30296522e-02,  4.42313291e-02,  4.16204929e-02,
        4.23112363e-02,  4.88363653e-02,  8.75333771e-02, -7.99607709e-02,
        7.72384852e-02,  2.95314975e-02,  6.16940670e-02, -5.78401126e-02,
       -5.84006310e-02, -1.26363738e-02,  8.83425307e-03, -3.63034643e-02,
       -1.20135993e-01, -1.94899328e-02,  1.41431959e-02,  6.60166368e-02,
       -1.84771139e-02, -9.75821093e-02,  1.92146394e-02,  1.38056492e-02,
       -4.73943427e-02, -5.04797287e-02,  3.20515297e-02, -2.33956613e-02,
       -2.17853617e-02,  4.21055295e-02,  1.76315196e-02, -3.94047201e-02,
        5.87634780e-02, -3.00273951e-02, -1.19601816e-01, -6.64142072e-02,
        6.66960434e-04,  7.31148720e-02, -9.36489925e-02,  7.28975423e-03,
        6.96717277e-02, -

In [36]:
class RatingsDataset(Dataset):
    def __init__(self, X, y):
        self.userId = X['userId']
        self.movieId = X['movieId']
        self.rating = y['rating']

    def __len__(self):
        return len(self.userId)

    def __getitem__(self, idx):
        userId     = self.userId.iloc[idx]
        movieId    = self.movieId.iloc[idx]
        abstracts  = movie2abstract_embed[movieId]
        rating     = self.rating.iloc[idx]
        movieId = movieid2idx.get(movieId, 0)
        userId = userid2idx.get(movieId, 0)

        return userId, movieId, abstracts, rating

In [37]:
train_data = RatingsDataset(X_train, y_train)
val_data = RatingsDataset(X_val, y_val)
test_data = RatingsDataset(X_test, y_test)

In [38]:
train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=64, shuffle=False)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=False)

In [39]:
def train_step(model,
               data_generator,
               loss_fn,
               optimizer,
               device):

    model.train()

    train_loss = []

    for idx, (X_user, X_movie, X_abstracts, y) in enumerate(data_generator):
        y = y.view(-1, 1).float()
        X_user, X_movie, X_abstracts, y = X_user.to(device), X_movie.to(device), X_abstracts.to(device), y.to(device)


        y_pred = model(X_user, X_movie, X_abstracts)
        loss = loss_fn(y_pred, y)
        train_loss.append(loss.item())

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

        if idx % 25000 == 0:
            print('Step', idx)



    return np.mean(train_loss)

def test_step(model,
              data_generator,
              loss_fn,
              device):
    model.eval()

    test_loss = []

    with torch.inference_mode():
        for X_user, X_movie, X_abstracts, y in data_generator:
            y = y.view(-1, 1).float()
            X_user, X_movie, X_abstracts, y = X_user.to(device), X_movie.to(device), X_abstracts.to(device), y.to(device)


            y_pred = model(X_user, X_movie, X_abstracts)

            loss = loss_fn(y_pred, y)
            test_loss.append(loss.item())

    return np.mean(test_loss)

def train(model,
          train_data_generator,
          test_data_generator,
          optimizer,
          loss_fn,
          epochs,
          device,
          scheduler):

    results = {"train_loss": [],
               "test_loss": [],
    }

    model.to(device)

    best_loss = float('inf')

    for epoch in tqdm(range(epochs)):

        start_time = timer()
        train_loss = train_step(model=model,
                                          data_generator=train_data_generator,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
        end_time = timer()
        test_loss = test_step(model=model,
          data_generator=test_data_generator,
          loss_fn=loss_fn,
          device=device)
        scheduler.step(test_loss)

        if best_loss > test_loss:
            best_loss = test_loss
            print('Saving new best model.')
            model.save( output_path + model.name + ".pt")

        results["train_loss"].append(train_loss)
        results["test_loss"].append(test_loss)


        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"time: {(end_time-start_time):.4f} "
          f"Learning rate: {next(iter(scheduler.optimizer.param_groups))['lr']}"
        )

    return results

def plot_loss_curves(results):

    loss = results["train_loss"]
    test_loss = results["test_loss"]

    epochs = range(len(results["train_loss"]))

    plt.figure(figsize=(15, 7))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, label="train_loss")
    plt.plot(epochs, test_loss, label="test_loss")
    plt.title("Loss")
    plt.xlabel("Epochs")
    plt.legend()

In [40]:
class MF_FNN(nn.Module):
    def __init__(self, num_users, num_movies, abstracts_dim=384, emb_size=32, n_hidden=32, name='model'):
        super(MF_FNN, self).__init__()
        self.user_emb = nn.Embedding(num_users, emb_size)
        self.movie_emb = nn.Embedding(num_movies, emb_size)
        self.lin1 = nn.Linear(emb_size*2 + abstracts_dim, n_hidden)
        self.lin2 = nn.Linear(n_hidden, 1)
        self.drop1 = nn.Dropout(0.5)
        self.name = name

        self.user_emb.weight.data.uniform_(0, 0.05)
        self.movie_emb.weight.data.uniform_(0, 0.05)
        # self.user_emb.data = nn.Parameter(
        #     torch.Tensor(np.random.randn(num_users, emb_size) * 0.01)
        # )
        # self.movie_emb.data = nn.Parameter(
        #     torch.Tensor(np.random.randn(num_movies, emb_size) * 0.01)
        # )

    def save(self, fileName):
        torch.save(self.state_dict(), fileName)

    def load(self, fileName, device):
        self.load_state_dict(torch.load(fileName, map_location=device))

    def forward(self, user, movie, abstracts):
        U = self.user_emb(user)
        M = self.movie_emb(movie)
        x = F.relu(torch.cat([U, M, abstracts], dim=1))
        x = self.drop1(x)
        x = F.relu(self.lin1(x))
        x = self.lin2(x)
        return x

In [62]:
class RMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()

    def forward(self, yhat, y):
        return torch.sqrt(self.mse(yhat ,y))

In [63]:
model = MF_FNN(len(userid2idx), len(movieid2idx), emb_size=20, n_hidden=256, name="deep_mf_with_abstacts_embeddings_v1") # best 0.9504 0.9491
loss = RMSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=1, threshold=0.0001, threshold_mode='rel')

  0%|          | 0/20 [00:00<?, ?it/s]

Step 0
Saving new best model.
Epoch: 1 | train_loss: 1.0622 | test_loss: 0.9614 | time: 3.9852 Learning rate: 0.001
Step 0
Saving new best model.
Epoch: 2 | train_loss: 0.9787 | test_loss: 0.9590 | time: 4.4241 Learning rate: 0.001
Step 0
Epoch: 3 | train_loss: 0.9670 | test_loss: 0.9614 | time: 4.5806 Learning rate: 0.001
Step 0
Saving new best model.
Epoch: 4 | train_loss: 0.9584 | test_loss: 0.9565 | time: 3.9283 Learning rate: 0.001
Step 0
Saving new best model.
Epoch: 5 | train_loss: 0.9514 | test_loss: 0.9550 | time: 4.5082 Learning rate: 0.001
Step 0
Saving new best model.
Epoch: 6 | train_loss: 0.9477 | test_loss: 0.9515 | time: 4.4757 Learning rate: 0.001
Step 0
Saving new best model.
Epoch: 7 | train_loss: 0.9467 | test_loss: 0.9509 | time: 4.0093 Learning rate: 0.001
Step 0
Epoch: 8 | train_loss: 0.9444 | test_loss: 0.9523 | time: 4.5966 Learning rate: 0.001
Step 0
Saving new best model.
Epoch: 9 | train_loss: 0.9424 | test_loss: 0.9503 | time: 4.3654 Learning rate: 0.001
St

In [64]:
results = train(model=model,
          train_data_generator = train_dataloader,
          test_data_generator = val_dataloader,
          optimizer = optimizer,
          loss_fn = loss,
          epochs = 20,
          device = device,
          scheduler=scheduler)
plot_loss_curves(results)

In [65]:
class MF_FNN2(nn.Module):
    def __init__(self, num_users, num_movies, abstracts_dim=384, emb_size=32, n_hidden=32, name="model"):
        super(MF_FNN2, self).__init__()
        self.user_emb = nn.Embedding(num_users, emb_size)
        self.movie_emb = nn.Embedding(num_movies, emb_size)
        self.lin1 = nn.Linear(emb_size*2 + abstracts_dim, n_hidden)
        self.lin2 = nn.Linear(n_hidden, n_hidden)
        self.lin3 = nn.Linear(n_hidden, 1)
        self.drop1 = nn.Dropout(0.5)
        self.drop2 = nn.Dropout(0.5)
        self.name = name

        self.user_emb.weight.data.uniform_(0, 0.05)
        self.movie_emb.weight.data.uniform_(0, 0.05)
        # self.user_emb.data = nn.Parameter(
        #     torch.Tensor(np.random.randn(num_users, emb_size) * 0.01)
        # )
        # self.movie_emb.data = nn.Parameter(
        #     torch.Tensor(np.random.randn(num_movies, emb_size) * 0.01)
        # )
    def save(self, fileName):
        torch.save(self.state_dict(), fileName)

    def load(self, fileName, device):
        self.load_state_dict(torch.load(fileName, map_location=device))

    def forward(self, user, movie, abstracts):
        U = self.user_emb(user)
        M = self.movie_emb(movie)
        x = F.relu(torch.cat([U, M, abstracts], dim=1))
        x = F.relu(self.lin1(x))
        x = self.drop1(x)
        x = F.relu(self.lin2(x)) + x
        x = self.drop2(x)
        x = self.lin3(x)
        return x

  0%|          | 0/20 [00:00<?, ?it/s]

Step 0
Saving new best model.
Epoch: 1 | train_loss: 1.1630 | test_loss: 0.9990 | time: 4.3016 Learning rate: 0.001
Step 0
Saving new best model.
Epoch: 2 | train_loss: 1.0314 | test_loss: 0.9633 | time: 4.2843 Learning rate: 0.001
Step 0
Saving new best model.
Epoch: 3 | train_loss: 1.0021 | test_loss: 0.9628 | time: 5.4099 Learning rate: 0.001
Step 0
Saving new best model.
Epoch: 4 | train_loss: 0.9918 | test_loss: 0.9563 | time: 4.2852 Learning rate: 0.001
Step 0
Saving new best model.
Epoch: 5 | train_loss: 0.9807 | test_loss: 0.9547 | time: 4.3851 Learning rate: 0.001
Step 0
Epoch: 6 | train_loss: 0.9740 | test_loss: 0.9612 | time: 5.2932 Learning rate: 0.001
Step 0
Epoch: 7 | train_loss: 0.9663 | test_loss: 0.9616 | time: 4.2615 Learning rate: 0.0001
Step 0
Saving new best model.
Epoch: 8 | train_loss: 0.9531 | test_loss: 0.9539 | time: 4.8447 Learning rate: 0.0001
Step 0
Saving new best model.
Epoch: 9 | train_loss: 0.9490 | test_loss: 0.9535 | time: 4.7960 Learning rate: 0.0001

In [66]:
model = MF_FNN2(len(userid2idx), len(movieid2idx), emb_size=20, n_hidden=128, name="deep_mf_with_abstacts_embeddings_v2") # best 0.9525
loss = RMSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=1, threshold=0.0001, threshold_mode='rel')
results = train(model=model,
          train_data_generator = train_dataloader,
          test_data_generator = val_dataloader,
          optimizer = optimizer,
          loss_fn = loss,
          epochs = 20,
          device = device,
          scheduler=scheduler)
plot_loss_curves(results)